In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
cd /content/drive/MyDrive/2022-1/캡스톤/comebackhome

/content/drive/MyDrive/2022-1/캡스톤/comebackhome


In [3]:
!pip install --upgrade google-cloud-translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. 번역기 생성

In [4]:
from google.cloud import translate_v3beta1 as translate

In [5]:
def get_googlecloud_tranlator(project_id, location, service_key, return_parent = True):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_key
    parent = f"projects/{project_id}/locations/{location}"
    client = translate.TranslationServiceClient()
    if return_parent :
        return parent, client
    return client

In [17]:
def back_translate(x, client, parent, dest):
    if not x:
        return x

    response = client.translate_text(
                request = {
                    "parent": parent,
                    "contents" : [x],
                    "mime_type" : "text/plain",
                    "source_language_code" : "ko", 
                    "target_language_code" : dest,
                    }
            )
    x = response.translations[0].translated_text
    response = client.translate_text(
                request = {
                    "parent": parent,
                    "contents" : [x],
                    "mime_type" : "text/plain",
                    "source_language_code" : dest, 
                    "target_language_code" : "ko"
                    }
            )
    x = response.translations[0].translated_text
    return x

In [7]:
key = 'snappy-lattice-352109-c99183be3c3c.json'
project_id =  "snappy-lattice-352109"
location = "global"
parent, translator = get_googlecloud_tranlator(project_id, location, key)

# 2. back translation을 이용한 Data Augmentation
참고문헌 : Sergey Edunov et al. 2018. Understanding Back-Translation at Scale. 2018. arXiv 1808.09381 

#### 샘플이 5개 이하인 클래스의 트레인데이터만 증강

In [10]:
df = pd.read_csv("dataset_ver2.csv")

In [11]:
df = df.fillna("")

In [12]:
target_counts = df['level5/diagnosis'].value_counts().reset_index()
target_counts.columns = ['diagnosis', 'counts']
under_5 = target_counts.loc[target_counts.counts <= 5, 'diagnosis'].unique()

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('level5/diagnosis', axis = 1), df['level5/diagnosis'], random_state = 0, stratify = df['level5/diagnosis']) 

X_train.to_csv("X_train.csv", index = False)
X_test.to_csv("X_test.csv", index = False)
y_train.to_csv("y_train.csv", index = False)
y_test.to_csv("y_test.csv", index = False)

In [15]:
X_train_trans = X_train.loc[[y in under_5 for y in y_train]]
y_trans = y_train.loc[[y in under_5 for y in y_train['level5/diagnosis']]].reset_index(drop = True)

#### 영어, 독일어, 중국어, 일본어, 스페인어를 이용하여 back translate

In [ ]:
X_train_eng = X_train_trans.copy()
X_train_ger = X_train_trans.copy()
X_train_china = X_train_trans.copy() #zh-cn
X_train_ja = X_train_trans.copy()
X_train_spanish = X_train_trans.copy()  #es

In [ ]:
trans_features = ['Chief complaint', 'Onset', 'Location', 'Duration', 'Course', 'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '사회력', '외상력']
for f in trans_features:
    X_train_eng[f] = X_train_eng[f].apply(back_translate, args = (translator, parent, "en",))
    X_train_ger[f] = X_train_ger[f].apply(back_translate, args = (translator, parent, "ger",))
    X_train_china[f] = X_train_china[f].apply(back_translate, args = (translator, parent, "zh_cn",))
    X_train_ja[f] = X_train_ja[f].apply(back_translate, args = (translator, parent, "ja",))
    X_train_spanish[f] = X_train_spanish[f].apply(back_translate, args = (translator, parent, "es",))

In [ ]:
X_train = pd.concat([X_train, y_train], axis = 1)
X_train_ger = pd.concat([X_train_ger, y_trans], axis = 1)
X_train_china = pd.concat([X_train_china, y_trans], axis = 1)
X_train_ja = pd.concat([X_train_ja, y_trans], axis = 1)
X_train_spanish = pd.concat([X_train_spanish, y_trans], axis = 1)

In [ ]:
X_train = pd.concat([X_train, X_train_ger, X_train_china, X_train_ja, X_train_spanish]).reset_index(drop = True)

In [ ]:
X_train = X_train.sample(frac = 1)

In [ ]:
y_train_trans = X_train['level5/diagnosis']
y_train_trans.to_csv("y_train_trans.csv", index = False)

X_train_trans = X_train.drop('level5/diagnosis', axis = 1)
X_train_trans.to_csv("X_train_trans.csv", index = False)